In [33]:
import pandas as pd

flights_df = pd.read_csv('flights.csv',low_memory=False)
airports_df = pd.read_csv('airports.csv')
airlines_df = pd.read_csv('airlines.csv')

In [2]:
flights_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
airports_df.head()

,airport_code,airport,city,state,country,latitude,longitude
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [4]:
airlines_df.head()

,airline_code,airline
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


# Airport Scoring

## Scoring for quantity of flights per airport

In [130]:
airport_cancellations = pd.read_csv('airport_cancellations.csv')

total_flights_by_airport = airport_cancellations[['origin_airport', 'flights_out_of', 'flights_into']].copy()
total_flights_by_airport['flight_out_of_score'] = pd.qcut(total_flights_by_airport['flights_out_of'], q=100, labels=False, duplicates='drop') + 1
total_flights_by_airport['flights_into_score'] = pd.qcut(total_flights_by_airport['flights_into'], q=100, labels=False, duplicates='drop') + 1

total_flights_by_airport = total_flights_by_airport.sort_values('flights_out_of', ascending=False)

total_flights_by_airportdf = total_flights_by_airport.rename(columns={'origin_airport': 'airport', 'flights_out_of': 'flights_out_of_score', 'flights_into': 'flights_into_score'}).reset_index(drop=True)

In [131]:
total_flights_by_airport.head()

,origin_airport,flights_out_of,flights_into,flight_out_of_score,flights_into_score
0,Hartsfield-Jackson Atlanta International Airport,346836,346904,100,100
1,Chicago O'Hare International Airport,285884,285906,100,100
2,Dallas/Fort Worth International Airport,239551,239582,100,100
3,Denver International Airport,196055,196010,100,100
4,Los Angeles International Airport,194673,194696,99,99


In [141]:
total_flights_by_airport.to_csv('total_flights_by_airport_score')

## Scoring for number of airlines offered per airport

In [136]:
airline_airport_merge = flights_df.merge(airports_df, left_on='ORIGIN_AIRPORT', right_on='airport_code')
airline_airport_grouped = airline_airport_merge.groupby('airport').agg({'AIRLINE': 'nunique'})
airline_airport_grouped.rename(columns={'AIRLINE': 'airlines_quantity'}, inplace=True)
airline_airport_grouped['num_airlines_score'] = airline_airport_grouped['airlines_quantity'].transform(lambda x: pd.cut(x, bins=100, labels=False) + 1)
airline_airport_sorted = airline_airport_grouped.sort_values(by='airlines_quantity', ascending=False)

In [137]:
sorted_df.head(10)

,airlines_quantity,num_airlines_score
airport,,
Los Angeles International Airport,13,100
LaGuardia Airport (Marine Air Terminal),12,92
Louis Armstrong New Orleans International Airport,12,92
Chicago O'Hare International Airport,12,92
Ronald Reagan Washington National Airport,12,92
Philadelphia International Airport,12,92
San Diego International Airport (Lindbergh Field),12,92
Phoenix Sky Harbor International Airport,12,92
Portland International Airport,12,92


In [140]:
airline_airport_sorted.to_csv('total_airlines_by_airport_score.csv')

In [9]:
airline_airport_sorted.value_counts('num_airlines_score')

num_airlines_score
1     106
2      43
4      37
5      31
9      17
7      15
14     15
17     15
12     13
15     10
19     10
10      9
20      1
dtype: int64

In [10]:
airline_airport_sorted.value_counts('airlines_quantity')

airlines_quantity
1     106
2      43
3      37
4      31
6      17
5      15
9      15
11     15
8      13
10     10
12     10
7       9
13      1
dtype: int64

In [169]:
merged_df = flights_df.merge(airports_df, left_on='DESTINATION_AIRPORT', right_on='airport_code')
grouped_df = merged_df.groupby('airport').agg({'AIRLINE': 'nunique'})
grouped_df.rename(columns={'AIRLINE': 'airlines_quantity'}, inplace=True)
grouped_df['num_airlines_score'] = grouped_df['airlines_quantity'].transform(lambda x: pd.cut(x, bins=100, labels=False) + 1)
sorted_df = grouped_df.sort_values(by='airlines_quantity', ascending=False)

In [170]:
sorted_df.head(10)

,airlines_quantity,num_airlines_score
airport,,
Los Angeles International Airport,13,100
LaGuardia Airport (Marine Air Terminal),12,92
Louis Armstrong New Orleans International Airport,12,92
Chicago O'Hare International Airport,12,92
Ronald Reagan Washington National Airport,12,92
Philadelphia International Airport,12,92
San Diego International Airport (Lindbergh Field),12,92
Phoenix Sky Harbor International Airport,12,92
Portland International Airport,12,92


## Score for cancellation rate per airport

In [142]:
airport_cancellations = pd.read_csv('airport_cancellations.csv')
airport_cancellations['cancellation_rate_out_of'] = airport_cancellations['canceled_out_of'] / airport_cancellations['flights_out_of'] * 100
airport_cancellations['cancellation_rate_into'] = airport_cancellations['canceled_into'] / airport_cancellations['flights_into'] * 100
airport_cancellations['cancel_out_of_score'] = airport_cancellations['cancellation_rate_out_of'].rank(method='min', ascending=False, pct=True) * 100
airport_cancellations['cancel_into_score'] = airport_cancellations['cancellation_rate_into'].rank(method='min', ascending=False, pct=True) * 100
airport_cancellations = airport_cancellations[['origin_airport', 'flights_out_of', 'canceled_out_of', 'cancellation_rate_out_of', 'cancel_out_of_score', 'flights_into', 'canceled_into', 'cancellation_rate_into', 'cancel_into_score']]
airport_cancellations = airport_cancellations.sort_values('cancel_out_of_score', ascending=False)

In [143]:
df.head(5)

,origin_airport,flights_out_of,canceled_out_of,cancellation_rate_out_of,cancel_out_of_score,flights_into,canceled_into,cancellation_rate_into,cancel_into_score
288,Cedar City Regional Airport,573,1.0,0.174520,100.000000,573,1.0,0.174520,98.734177
106,Hilo International Airport,5723,12.0,0.209680,99.683544,5723,10.0,0.174734,98.417722
207,Daytona Beach International Airport,1494,4.0,0.267738,99.367089,1494,4.0,0.267738,97.784810
274,Pocatello Regional Airport,654,2.0,0.305810,99.050633,654,2.0,0.305810,96.835443
232,Henry E. Rohlsen Airport,939,3.0,0.319489,98.734177,938,3.0,0.319829,96.518987


In [144]:
airport_cancellations.to_csv('cancellation_rate_by_airport_score.csv')

## Score for on-time by airport

In [145]:
t1 = flights[flights['SCHEDULED_ARRIVAL'] - flights['ARRIVAL_TIME'] <= 15]\
    .groupby('ORIGIN_AIRPORT').size().reset_index(name='count_on_time')
t2 = flights.groupby('ORIGIN_AIRPORT').size().reset_index(name='total_count')
result = pd.merge(t1, t2, on='ORIGIN_AIRPORT')
result = pd.merge(result, airports_df[['airport_code', 'airport']], left_on='ORIGIN_AIRPORT', right_on='airport_code')
result['on_time_percent'] = round(result['count_on_time'] / result['total_count'] * 100, 2)
result['on_time_score'] = pd.qcut(result['on_time_percent'], q=100, labels=False, duplicates='drop').add(1)
result = result.sort_values(by='on_time_percent', ascending=False)
result = result[['airport', 'count_on_time', 'total_count', 'on_time_percent', 'on_time_score']]

In [146]:
result.head()

,airport,count_on_time,total_count,on_time_percent,on_time_score
246,Pago Pago International Airport (Tafuna Airport),103,107,96.26,100
162,Hilo International Airport,5275,5723,92.17,100
13,King Salmon Airport,58,63,92.06,100
63,McClellan-Palomar Airport,573,641,89.39,100
27,Bethel Airport,790,896,88.17,99


In [147]:
result.to_csv('on_time_by_airport_score.csv')

In [167]:
t1 = flights[flights['SCHEDULED_ARRIVAL'] - flights['ARRIVAL_TIME'] <= 15]\
    .groupby('DESTINATION_AIRPORT').size().reset_index(name='count_on_time')
t2 = flights.groupby('DESTINATION_AIRPORT').size().reset_index(name='total_count')
result = pd.merge(t1, t2, on='DESTINATION_AIRPORT')
result = pd.merge(result, airports_df[['airport_code', 'airport']], left_on='DESTINATION_AIRPORT', right_on='airport_code')
result['on_time_percent'] = round(result['count_on_time'] / result['total_count'] * 100, 2)
result['on_time_score'] = pd.qcut(result['on_time_percent'], q=100, labels=False, duplicates='drop').add(1)
result = result.sort_values(by='on_time_percent', ascending=False)
result = result[['DESTINATION_AIRPORT', 'count_on_time', 'total_count', 'on_time_percent', 'on_time_score']]

In [168]:
result.to_csv('on_time_by_destination_airport_score.csv')

## Scoring for avg delay time per airport

In [148]:
merged_df = pd.merge(flights_df, airports_df, left_on='ORIGIN_AIRPORT', right_on='airport_code')
grouped_df = merged_df.groupby(['ORIGIN_AIRPORT']).agg(avg_delay=('ARRIVAL_DELAY', 'mean')).reset_index()
grouped_df['score'] = pd.qcut(grouped_df['avg_delay'], 100, labels=False, duplicates='drop')
grouped_df['score'] = grouped_df['score'].max() - grouped_df['score'] + 1
grouped_df = grouped_df.sort_values(by=['avg_delay'], ascending=False)
result_df = grouped_df[['ORIGIN_AIRPORT', 'avg_delay', 'score']]
result_df['avg_delay'] = result_df['avg_delay'].round(2)

In [149]:
result_df

,ORIGIN_AIRPORT,avg_delay,score
154,ILG,24.06,1
133,GST,21.76,1
246,PPG,21.50,1
293,STC,19.56,1
40,BPT,16.15,2
...,...,...,...
46,BTM,-6.09,99
189,LWS,-6.62,100
314,VEL,-7.52,100
89,DLG,-7.87,100


In [150]:
result_df.to_csv('delay_by_airport_score.csv')

In [164]:
merged = pd.merge(flights_df, airports_df, left_on='DESTINATION_AIRPORT', right_on='airport_code')
grouped_df = merged.groupby(['DESTINATION_AIRPORT']).agg(avg_delay=('ARRIVAL_DELAY', 'mean')).reset_index()
grouped_df['score'] = pd.qcut(grouped_df['avg_delay'], 100, labels=False, duplicates='drop')
grouped_df['score'] = grouped_df['score'].max() - grouped_df['score'] + 1
grouped_df = grouped_df.sort_values(by=['avg_delay'], ascending=False)
result = grouped_df[['DESTINATION_AIRPORT', 'avg_delay', 'score']]
result['avg_delay'] = result_df['avg_delay'].round(2)

In [165]:
result

,DESTINATION_AIRPORT,avg_delay,score
293,STC,23.03,1
154,ILG,21.99,1
305,TTN,17.43,1
137,GUM,17.32,1
231,OTH,16.74,2
...,...,...,...
189,LWS,-6.54,99
35,BLI,-6.80,100
156,IMT,-7.60,100
70,CNY,-11.58,100


In [166]:
result.to_csv('delay_by_destination_airport_score.csv')

# Airline Scoring

## Scoring for percent of total flights by airline

In [178]:
airline_counts = flights.groupby('AIRLINE').size().reset_index(name='total_flights')
airline_counts = airline_counts.merge(airlines, left_on='AIRLINE', right_on='airline_code')
total_flights = len(flights)
airline_counts['percentage'] = airline_counts['total_flights'] / total_flights * 100
airline_counts['score'] = pd.qcut(airline_counts[['total_flights']].reset_index(drop=True)['total_flights'].values, 100, labels=False, duplicates='drop') + 1
airline_counts = airline_counts[['AIRLINE', 'total_flights', 'percentage', 'score']]
airline_counts = airline_counts.sort_values('total_flights', ascending=False)

In [180]:
airline_counts.head()

,AIRLINE,total_flights,percentage,score
13,WN,1261855,21.684789,100
3,DL,875881,15.051884,93
0,AA,725984,12.475926,85
9,OO,588353,10.110758,77
4,EV,571977,9.829339,70


In [152]:
airline_counts.to_csv('airline_counts_score.csv')

## Scoring for cancellation rate by airline

In [181]:
merged = pd.merge(flights_df, airlines_df, left_on='AIRLINE', right_on='airline_code')
grouped = merged.groupby('AIRLINE').agg({
    'CANCELLED': 'sum',
    'AIRLINE': 'count'
})
grouped['cancellation_rate'] = grouped['CANCELLED'] / grouped['AIRLINE'] * 100
grouped['score'] = grouped['cancellation_rate'].rank(pct=True, method='min', ascending=False).apply(lambda x: int(x*100))
result = grouped.rename(columns={'CANCELLED': 'cancelled_count', 'AIRLINE': 'total_count'}).sort_values('cancellation_rate', ascending=False)

In [182]:
result.head()

,cancelled_count,total_count,cancellation_rate,score
AIRLINE,,,,
MQ,15025,294632,5.099582,7
EV,15231,571977,2.662869,14
US,4067,198715,2.046650,21
NK,2004,117379,1.707290,28
OO,9960,588353,1.692861,35


In [172]:
result.to_csv('airline_cancellation_rate_score.csv')

## Scoring for on time rate per airline

In [183]:
t1 = flights_df[flights_df['SCHEDULED_ARRIVAL'] - flights_df['ARRIVAL_TIME'] <= 15].groupby('AIRLINE').size().reset_index(name='count_on_time')
t2 = flights_df.groupby('AIRLINE').size().reset_index(name='total_count')
merged_df = pd.merge(t1, t2, on='AIRLINE').merge(airlines_df, left_on='AIRLINE', right_on='airline_code')
merged_df['on_time_percent'] = round(merged_df['count_on_time'] / merged_df['total_count'] * 100, 2)
merged_df['on_time_score'] = pd.qcut(merged_df['on_time_percent'], 100, labels=False, duplicates='drop').add(1)
result = merged_df[['AIRLINE', 'count_on_time', 'total_count', 'on_time_percent', 'on_time_score']].sort_values(by='on_time_score', ascending=False)

In [184]:
result.head()

,AIRLINE,count_on_time,total_count,on_time_percent,on_time_score
6,HA,68145,76272,89.34,100
8,NK,91401,117379,77.87,93
9,OO,449987,588353,76.48,85
5,F9,69204,90836,76.19,77
13,WN,951781,1261855,75.43,70


In [157]:
result.to_csv('airline_on_time_score.csv')

## Score for average delay by airline

In [193]:
avg_delay = pd.merge(flights, airlines, left_on='AIRLINE', right_on='airline_code')
grouped_df = avg_delay.groupby(['AIRLINE']).agg(avg_delay=('ARRIVAL_DELAY', 'mean')).reset_index()
grouped_df['score'] = pd.qcut(grouped_df['avg_delay'], 100, labels=False, duplicates='drop')
grouped_df['score'] = grouped_df['score'].max() - grouped_df['score'] + 1
grouped_df = grouped_df.sort_values(by=['avg_delay'], ascending=False)
result = grouped_df[['AIRLINE', 'avg_delay', 'score']]
result['avg_delay'] = result_df['avg_delay'].round(2)

In [194]:
result.head()

,AIRLINE,avg_delay,score
8,NK,14.47,1
5,F9,12.50,8
2,B6,6.68,16
4,EV,6.59,24
7,MQ,6.46,31


In [163]:
result.to_csv('airline_avg_delay_score.csv')

## Scoring for number of airports by airlines

In [173]:
merged_df = pd.merge(flights_df, airlines_df, left_on='AIRLINE', right_on='airline_code')

grouped_df = merged_df.groupby(['AIRLINE']).agg(num_origin_airports=('ORIGIN_AIRPORT', 'nunique')).reset_index()

grouped_df['score'] = pd.qcut(grouped_df['num_origin_airports'], 100, labels=False, duplicates='drop') + 1
grouped_df = grouped_df.sort_values(by=['num_origin_airports'], ascending=False)

result_df = grouped_df[['AIRLINE', 'num_origin_airports', 'score']]

In [176]:
result_df.to_csv('airline_num_of_airports.csv')

## Scoring for number of airports traveled into per airport

In [228]:
total_airports = pd.merge(flights_df, airports_df, left_on='ORIGIN_AIRPORT', right_on='airport_code')
total_airports_grouped = total_airports.groupby('ORIGIN_AIRPORT').agg({'DESTINATION_AIRPORT': pd.Series.nunique}).reset_index()
total_airports_grouped = total_airports_grouped.sort_values(by='DESTINATION_AIRPORT', ascending=False)
total_airports_grouped['total_airports_score'] = pd.qcut(total_airports_grouped['DESTINATION_AIRPORT'], q=379, labels=False, duplicates='drop') + 1
total_airports_sorted = total_airports_grouped.sort_values(by='total_airports_score', ascending=False)

In [229]:
total_airports_sorted.head()

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,total_airports_score
20,ATL,169,100
228,ORD,162,99
86,DFW,148,98
85,DEN,139,97
216,MSP,120,96


In [231]:
total_airports_sorted.to_csv('total_airports_score.csv')

## Iterable code for airline scoring system

In [177]:
import pandas as pd

def airline_scores(month, year):
    # import data
    flights_df = pd.read_csv(f'flights_{year}_{month}.csv')
    airlines_df = pd.read_csv('airlines.csv')
    
    # merge files
    merged_df = pd.merge(flights_df, airlines_df, left_on='AIRLINE', right_on='airline_code')
    
    # calculating scores
    count_unique = merged_df.groupby(['AIRLINE']).agg(num_origin_airports=('ORIGIN_AIRPORT', 'nunique')).reset_index()
    count_unique['score'] = pd.qcut(grouped_df['num_origin_airports'], 100, labels=False, duplicates='drop') + 1
    count_unique = count_unique.sort_values(by=['num_origin_airports'], ascending=False)
    result_df = count_unique[['AIRLINE', 'num_origin_airports', 'score']]
    
    airline_counts = flights_df.groupby('AIRLINE').size().reset_index(name='total_flights')
    airline_counts = airline_counts.merge(airlines_df, left_on='AIRLINE', right_on='airline_code')
    total_flights = len(flights_df)
    airline_counts['percentage'] = airline_counts['total_flights'] / total_flights * 100
    airline_counts['score'] = pd.qcut(airline_counts[['total_flights']].reset_index(drop=True)['total_flights'].values, 100, labels=False, duplicates='drop') + 1
    airline_counts = airline_counts[['AIRLINE', 'total_flights', 'percentage', 'score']]
    airline_counts = airline_counts.sort_values('total_flights', ascending=False)

    avg_delay = merged_df.groupby(['AIRLINE']).agg(avg_delay=('ARRIVAL_DELAY', 'mean')).reset_index()
    avg_delay['score'] = pd.qcut(avg_delay['avg_delay'], 100, labels=False, duplicates='drop')
    avg_delay['score'] = avg_delay['score'].max() - avg_delay['score'] + 1
    avg_delay = avg_delay.sort_values(by=['avg_delay'], ascending=False)
    delay_df = avg_delay[['AIRLINE', 'avg_delay', 'score']]
    delay_df['avg_delay'] = result_df['avg_delay'].round(2)
    
    on_time = flights_df[flights_df['SCHEDULED_ARRIVAL'] - flights_df['ARRIVAL_TIME'] <= 15].groupby('AIRLINE').size().reset_index(name='count_on_time')
    on_time_group = flights_df.groupby('AIRLINE').size().reset_index(name='total_count')
    merged_df = pd.merge(on_time, on_time_group, on='AIRLINE').merge(airlines_df, left_on='AIRLINE', right_on='airline_code')
    merged_df['on_time_percent'] = round(merged_df['count_on_time'] / merged_df['total_count'] * 100, 2)
    merged_df['on_time_score'] = pd.qcut(merged_df['on_time_percent'], 100, labels=False, duplicates='drop').add(1)
    result_df = merged_df[['AIRLINE', 'count_on_time', 'total_count', 'on_time_percent', 'on_time_score']].sort_values(by='on_time_score', ascending=False)
    merged_df = pd.merge(flights, airlines, left_on='AIRLINE', right_on='airline_code')